# Homework

The idea of this homework is to play around with different prompt techniques, depending on the input data to the LLM.
Some of the prompt techniques that are important to grasp are:
- Code Generation
- Question Answering
- Text Summarization
- Reasoning
- Chain of Thoughts
- Zero-shot prompting
- Few-shot prompting

Make multiple prompt examples for each of the techniques above. Try to use different models and model settings.


In [1]:
# Installing the required packages
!pip3 install langchain langchain_community boto3

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Importing the required packages
import boto3
import botocore
import json
from langchain_community.chat_models import BedrockChat

/Users/raquelcardoso/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in BedrockBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/raquelcardoso/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in BedrockBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


If you prefer, you can also use bedrock native functions from the bedrock_setup notebook

In [3]:
bedrock_client = boto3.client("bedrock-runtime", "us-east-1")  # region has to be this

In [4]:
bedrock_client._endpoint

bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

In [5]:
bedrock_client

In [6]:
def call_model(prompt_data, model):
    body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
    modelId = model  # change this to use a different version from the model provider
    accept = "application/json"
    contentType = "application/json"

    try:
        response = bedrock_client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        print(response_body.get("completion"))

    except botocore.exceptions.ClientError as error:
        if error.response["Error"]["Code"] == "AccessDeniedException":
            print(
                f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                    \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                    \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n"
            )

        else:
            raise error

In [7]:
claude = "anthropic.claude-v2"
titan = "amazon.titan-embed-text-v1"

# Code Generation

In [8]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data ="""Human:\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\"

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Here is a MySQL query to get all students in the Computer Science department:

```sql
SELECT s.StudentId, s.StudentName 
FROM students s
INNER JOIN departments d ON s.DepartmentId = d.DepartmentId
WHERE d.DepartmentName = 'Computer Science';
```

This performs an inner join between the students and departments tables, matching on the DepartmentId column. It then selects the StudentId and StudentName columns from the students table, and filters the results to only include rows where the DepartmentName is 'Computer Science'.

The inner join connects the two tables so we can filter based on the department name from the departments table, while selecting the student information from the students table.


In [9]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data ="""Human:Generate a Python script that reads a CSV file, filters the rows based on a specific column's value, and saves the filtered result into a new CSV file. The script should:
1. Use pandas library for reading and writing CSV files.
2. Filter rows where the value in the 'Status' column is 'Active'.
3. Save the filtered rows into a file named 'filtered_output.csv'.

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Here is the Python script to filter a CSV file based on a column value and save the result to a new CSV file:

```python
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv('input.csv') 

# Filter rows where Status is Active
filtered_df = df[df['Status'] == 'Active']

# Save filtered DataFrame to new CSV file
filtered_df.to_csv('filtered_output.csv', index=False)
```

To use it:

- Place the input CSV file in the same directory and name it `input.csv`
- The filtered output will be saved as `filtered_output.csv` 
- Update the column name if needed - the filter is on the `'Status'` column
- Set the filter value to match the desired rows - it's filtering for `'Active'` rows.

This will load the CSV, filter for rows where Status is 'Active', and save those filtered rows to a new CSV file.


# Question Answering

In [10]:
prompt_data = """System: Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Human: This is the question:

<question>
What was OKT3 originally sourced from?
</question>

This is the context:

<context> 
Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use. 
</context>

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Based on the context, OKT3 was originally sourced from mice.


In [11]:
prompt_data = """System: Answer the following question based on the context provided. If the answer is unclear, respond with "Unsure about answer." Keep your response short and to the point.

Human: This is the question:

<question>
What was the original source of penicillin?
</question>

This is the context:

<context>
Penicillin, the world's first antibiotic, was originally discovered by Scottish bacteriologist Alexander Fleming in 1928. He noticed that a mold called *Penicillium notatum* was secreting a substance that killed bacteria. This breakthrough revolutionized medicine, as penicillin became the most effective treatment against bacterial infections during World War II.
</context>

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Based on the provided context, the original source of penicillin was a mold called Penicillium notatum, which was first discovered by Alexander Fleming in 1928.


# Text Summarization

In [13]:
prompt_data = """Human: Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence:

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Antibiotics are medications that kill or stop the growth of bacteria, allowing the body's immune system to eliminate the infection, but they are ineffective against viruses and improper use can lead to resistant bacteria.


In [12]:
prompt_data = """Human: The Great Barrier Reef, located off the coast of Queensland, Australia, is the world's largest coral reef system. Comprising over 2,900 individual reefs and 900 islands, it spans over 2,300 kilometers. The reef supports a wide diversity of marine life and is a crucial ecosystem for the planet. However, in recent years, the reef has faced significant threats due to climate change, pollution, and coral bleaching. Conservation efforts are underway, but the future of the reef remains uncertain.

Explain the above in one sentence:

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 The Great Barrier Reef off Australia's Queensland coast is the world's largest coral reef system spanning over 2,300 kilometers with 2,900 reefs and 900 islands, supporting diverse marine life but facing threats from climate change, pollution, and bleaching.


In [14]:
prompt_data = """Human: Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above like I'm 5 years old:

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Here is an explanation of antibiotics for a 5 year old:

There are tiny living things called bacteria that can sometimes make us sick. When bacteria get inside our bodies where they don't belong, they can cause infections. Antibiotics are special medicines that help fight the bacteria and make us feel better.

Antibiotics work like little soldiers that attack the invading bacteria. Some antibiotics defeat the bacteria by poking holes in them until they die. Other antibiotics cover up the bacteria so they can't grow or make more bacteria. This allows the body's own defenses, called the immune system, to come in and get rid of the infection. 

Doctors prescribe antibiotics to treat illnesses caused by certain types of bacteria. We take the antibiotics by swallowing pills or liquid medicine. The medicine then goes inside our bodies and hunts down the bad bacteria. Antibiotics don't work on viruses - they only attack bacteria. It's important to take antibiotics exactly as the doctor instr

# Reasoning

In [15]:
prompt_data = """Human: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even.

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Okay, let's break this down step-by-step:

1) Identify the odd numbers in the group: 15, 5, 13, 7, 1

2) Add the odd numbers: 
15 + 5 + 13 + 7 + 1 = 41

3) The result, 41, is an odd number.

So the statement is false - the odd numbers in the group do not add up to an even number. The sum of the odd numbers is 41, which is an odd number.


In [16]:
prompt_data = """Human: In a class of 30 students, 18 are girls and the rest are boys. 

Solve by breaking the problem into steps. First, determine how many boys are in the class. Then, calculate the percentage of girls in the class.

Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Okay, here are the steps to solve this problem:

1) There are 30 students total in the class
2) 18 of the students are girls
3) So there must be 30 - 18 = 12 boys in the class
4) To calculate the percentage of girls:
- There are 18 girls 
- Out of a total of 30 students
- So the percentage of girls is (18/30) x 100% = 60%

Therefore, the number of boys is 12 and the percentage of girls is 60%.


# Chain of Thoughts

In [17]:
prompt_data = """Human: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
Assistant: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

Human: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Okay, let's go through the steps:
1) Identify the odd numbers in the group: 15, 5, 13, 7, 1
2) Add the odd numbers: 15 + 5 + 13 + 7 + 1 = 41
3) 41 is an odd number

Therefore, the statement "The odd numbers in this group add up to an even number" is false for this group of numbers.


In [18]:
prompt_data = """Human: The odd numbers in this group add up to an even number: 2, 4, 6, 7, 10, 11, 13.
Assistant: First, identify the odd numbers: 7, 11, 13. 
Now, add them: 7 + 11 + 13 = 31. 
Since 31 is odd, the statement is False.

Human: The odd numbers in this group add up to an even number: 1, 3, 5, 7, 8, 10, 12.
Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)

 Okay, let's analyze this new statement:

The odd numbers in the group are:
1, 3, 5, 7

Adding them up: 
1 + 3 + 5 + 7 = 16

16 is an even number. 

Therefore, the statement is true - the odd numbers in this group (1, 3, 5, 7) do add up to an even number (16).


# Zero-shot prompting

In [19]:
prompt_data = """Human: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step.
Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)


 Okay, here are the steps:

1) I originally bought 10 apples
2) I gave 2 apples to the neighbor 
3) I gave 2 apples to the repairman
4) So now I have 10 - 2 - 2 = 6 apples
5) I bought 5 more apples, so now I have 6 + 5 = 11 apples 
6) I ate 1 apple
7) So now I have 11 - 1 = 10 apples remaining

Therefore, the number of apples I remain with is 10.


In [20]:
prompt_data = """Human: Classify this review: 'The movie was very good.' Sentiment:
Assistant:
"""
call_model(prompt_data=prompt_data, model=claude)


 Okay, let's analyze this:

The review "The movie was very good" expresses a positive sentiment. Words like "very good" indicate the reviewer enjoyed the movie and had a positive experience. So I would classify this as a positive review.


# Few-shot prompting

In [22]:
prompt_data = """Human: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
Assistant: The answer is False.

Human: The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
Assistant: The answer is True.

Human: The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
Assistant: The answer is True.

Human: The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
Assistant: The answer is False.

Human: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
Assistant:"""
call_model(prompt_data=prompt_data, model=claude)

 Let's analyze this:
The odd numbers in the group are:
15
5
13
7
1

Adding them up:
15 + 5 + 13 + 7 + 1 = 41

41 is an odd number.

Therefore, the statement "The odd numbers in this group add up to an even number" is False.

So the answer is False.


In [21]:
prompt_data = """Human: Classify this review: 'The movie was very good.' Sentiment:
Assistant: Positive.

Human: Classify this review: 'It's just a waste of money.' Sentiment:
Assistant: Negative.

Human: Classify this review: 'Can't believe I wasted my time with this.' Sentiment:
Assistant: Negative.

Human: Classify this review: 'I would do it again.' Sentiment:
Assistant: Positive.

Human: Classify this review: 'It's useful but expensive.' Sentiment:
Assistant:"""
call_model(prompt_data=prompt_data, model=claude)

 This review expresses both positive and negative sentiments. The positive sentiment is that it is useful. The negative sentiment is that it is expensive. Overall I would classify this as a mixed sentiment review.
